In [1]:
"""
@author: brianwsyang
@date: Sat Jul 27, 2019
"""

import pandas as pd
import numpy as np
import re

In [18]:
crm = pd.read_csv('crime_data_w_population_and_crime_rate.csv')

In [19]:
crm

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,200,1778,3609,4995,13791,3543,464,318416,29,510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,38,165,662,1482,1753,189,28,49746,5,35
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,2,5,119,82,184,12,2,7629,17,3
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,3,1,2,5,4,4,0,412,48,261
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,4,17,368,149,494,60,0,27083,22,31
5,"Baltimore city, MD",1419.537569,6,1,4,1227,625474,625474,9,9,...,317,3638,4660,7804,18055,4009,251,622104,24,510
6,"Shelby County, TN",1315.003752,7,1,4,2534,942807,942807,15,14,...,483,3512,8224,14011,30620,3233,435,939465,47,157
7,"Marlboro County, SC",1253.437132,8,1,4,2377,28780,28780,9,8,...,20,32,296,408,1074,67,5,28003,45,69
8,"Storey County, NV",1217.656012,9,1,4,1778,3947,3947,1,1,...,7,2,39,23,55,8,2,3942,32,29
9,"District of Columbia, DC",1216.801325,10,1,4,325,632323,632323,2,1,...,236,4037,3505,3519,23575,3663,51,646449,11,1


In [3]:
cou_names = {}

for cou in crm.iloc[:,0]:
    cou_names[cou] = cou[:-4]

In [4]:
crm = crm.replace({'county_name': cou_names}).drop(['index', 'EDITION', 'PART'], axis=1)

In [5]:
cou_crm = crm[crm['county_name'].str.contains('County')]

In [13]:
cou_crm

,county_name,crime_rate_per_100000,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,INDEX,MODINDX,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
1,Crittenden County,1754.914968,130,50717,50717,4,4,100.0000,873,3424,...,38,165,662,1482,1753,189,28,49746,5,35
2,Alexander County,1664.700485,604,8040,8040,2,2,100.0000,127,278,...,2,5,119,82,184,12,2,7629,17,3
3,Kenedy County,1456.310680,2681,444,444,1,1,100.0000,6,13,...,3,1,2,5,4,4,0,412,48,261
6,Shelby County,1315.003752,2534,942807,942807,15,14,100.0000,12354,47864,...,483,3512,8224,14011,30620,3233,435,939465,47,157
7,Marlboro County,1253.437132,2377,28780,28780,9,8,100.0000,351,1549,...,20,32,296,408,1074,67,5,28003,45,69
8,Storey County,1217.656012,1778,3947,3947,1,1,100.0000,48,86,...,7,2,39,23,55,8,2,3942,32,29
10,Dillon County,1165.583272,2359,32060,32060,9,8,100.0000,364,1906,...,17,48,291,595,1206,105,12,31229,45,33
11,Scott County,1163.145839,1597,40941,40941,7,8,100.0000,457,1277,...,7,26,422,236,1002,39,8,39290,29,201
12,Philadelphia County,1150.296330,2320,1538957,1538957,6,5,100.0000,17866,57000,...,881,7984,8669,12004,38595,6401,0,1553165,42,101
13,Pershing County,1148.756725,1777,6822,6822,2,2,100.0000,79,75,...,3,1,75,27,43,5,0,6877,32,27


In [6]:
cou_avg = cou_crm.columns.tolist()

In [8]:
cou_avg

['county_name',
 'crime_rate_per_100000',
 'IDNO',
 'CPOPARST',
 'CPOPCRIM',
 'AG_ARRST',
 'AG_OFF',
 'COVIND',
 'INDEX',
 'MODINDX',
 'MURDER',
 'RAPE',
 'ROBBERY',
 'AGASSLT',
 'BURGLRY',
 'LARCENY',
 'MVTHEFT',
 'ARSON',
 'population',
 'FIPS_ST',
 'FIPS_CTY']

In [9]:
da_cou = cou_crm.max(axis=0).tolist()
da_cou[0] = 'Doña Ana County'
da_cou

['Doña Ana County',
 1754.914968,
 3177,
 9980757,
 9980757,
 150,
 140,
 100.0,
 44556,
 232266,
 600,
 1976,
 18923,
 23057,
 49107,
 142604,
 40555,
 2210,
 10017068,
 56,
 507]

In [17]:
da_county = ['Doña Ana County']

cou_crm.append(pd.DataFrame([da_cou], columns=cou_avg), ignore_index=True, sort=False)

,county_name,crime_rate_per_100000,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,INDEX,MODINDX,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,Crittenden County,1754.914968,130,50717,50717,4,4,100.0000,873,3424,...,38,165,662,1482,1753,189,28,49746,5,35
1,Alexander County,1664.700485,604,8040,8040,2,2,100.0000,127,278,...,2,5,119,82,184,12,2,7629,17,3
2,Kenedy County,1456.310680,2681,444,444,1,1,100.0000,6,13,...,3,1,2,5,4,4,0,412,48,261
3,Shelby County,1315.003752,2534,942807,942807,15,14,100.0000,12354,47864,...,483,3512,8224,14011,30620,3233,435,939465,47,157
4,Marlboro County,1253.437132,2377,28780,28780,9,8,100.0000,351,1549,...,20,32,296,408,1074,67,5,28003,45,69
5,Storey County,1217.656012,1778,3947,3947,1,1,100.0000,48,86,...,7,2,39,23,55,8,2,3942,32,29
6,Dillon County,1165.583272,2359,32060,32060,9,8,100.0000,364,1906,...,17,48,291,595,1206,105,12,31229,45,33
7,Scott County,1163.145839,1597,40941,40941,7,8,100.0000,457,1277,...,7,26,422,236,1002,39,8,39290,29,201
8,Philadelphia County,1150.296330,2320,1538957,1538957,6,5,100.0000,17866,57000,...,881,7984,8669,12004,38595,6401,0,1553165,42,101
9,Pershing County,1148.756725,1777,6822,6822,2,2,100.0000,79,75,...,3,1,75,27,43,5,0,6877,32,27


In [ ]:
# test to find the missing county
# missing county = Doña Ana County

In [ ]:
c_test = crm[crm['county_name'].str.contains('County')]['county_name']
c1_test = c_test.sort_values(ascending=True).reset_index()['county_name'].tolist()
c1_test

In [ ]:
test01 = pd.read_csv('counties.csv')
test02 = test01['County'].sort_values(ascending=True).reset_index()['County'].tolist()

In [ ]:
for t in np.arange(len(c1_test)):
    if c1_test[t] != test02[t]:
        print(test02[t])
        print(c1_test[t])
        break

In [ ]:
pd_test.to_csv('ree')

In [ ]:
test01[test01['County'].str.contains('Ana')]